In [10]:
# pip install tda-api

# from tda import auth, client
# import json

# import config_tda
# from tda import auth, client
# import json


# try:
#     c = auth.client_from_token_file(config_tda.token_path, config_tda.api_key)
# except FileNotFoundError:
#     from selenium import webdriver
#     with webdriver.Chrome() as driver:
#         c = auth.client_from_login_flow(
#             driver, api_key, redirect_uri, token_path)

# r = c.get_price_history('AAPL',
#         period_type=client.Client.PriceHistory.PeriodType.YEAR,
#         period=client.Client.PriceHistory.Period.TWENTY_YEARS,
#         frequency_type=client.Client.PriceHistory.FrequencyType.DAILY,
#         frequency=client.Client.PriceHistory.Frequency.DAILY)
# assert r.status_code == 200, r.raise_for_status()
# print(json.dumps(r.json(), indent=4))
# try:
#     c = auth.client_from_token_file(config_tda.token_path, config_tda.api_key)
# except FileNotFoundError:
#     from selenium import webdriver
#     with webdriver.Chrome() as driver:
#         c = auth.client_from_login_flow(
#             driver, api_key, redirect_uri, token_path)

# r = c.get_account(client.Client.Account.ORDERS)
# assert r.status_code == 200, r.raise_for_status()
# print(json.dumps(r.json(), indent=4))

In [11]:
import alpaca_trade_api as aplpaca
from alpaca_trade_api.rest import REST, TimeFrame
import os
from dotenv import load_dotenv

from web3 import Web3
from pathlib import Path
import json
load_dotenv()

True

In [12]:

# Set Alpca API key and secret key, then insantiate an API object
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECURITY_KEY")

tradeapi = REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [13]:
def get_ticker(ticker, start, end, tradeapi):
    """Executes the get_bars method from Alpaca Trade API"""
    return tradeapi.get_bars(
        ticker,
        TimeFrame.Day,
        start,
        end
    ).df
    
def grade_range(one_row_df):
    """Return the high and low of a single day"""
    return one_row_df[['high', 'low']].reset_index().drop(columns='timestamp').values

In [14]:
grade_range(get_ticker("SPY", "2022-07-26", "2022-07-26", tradeapi))

array([[394.06, 389.95]])

In [15]:
class Grader:
    # add event filters to Grader class 
    def __init__(self, trader_id, log_address):
        self.trader_id = trader_id
        self.log_address = log_address
        
    def load_contract(self):
        contract_address = self.log_address
        with open(Path('logger.json')) as f:
            logger_abi = json.load(f)
            
            self.contract = w3.eth.contract(
            address=contract_address,
            abi=logger_abi)
            
            self.new_trade_filter = self.contract.events.newTrade.createFilter(fromBlock="latest")
            self.close_trade_filter = self.contract.events.newCloseTrade.createFilter(fromBlock="latest")
            
        return None
    
    def verify_trade(self):
        pass        
    

In [16]:
import streamlit as st
import alpaca_trade_api as aplpaca
from alpaca_trade_api.rest import REST, TimeFrame
import os
import time
from dotenv import load_dotenv

from web3 import Web3
from pathlib import Path

import json
from io import StringIO

load_dotenv()


w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

def load_deployer_contract():
    contract_address = os.getenv("DEPLOYER_ADDRESS") 
    with open(Path('deployer.json')) as f:
            deployer_abi = json.load(f)

    deployer_contract = w3.eth.contract(
        address=contract_address,
        abi=deployer_abi
        )
    trader_dict = {}
    return deployer_contract, trader_dict

def load_contract_new(log_addr):
    contract_address = log_addr
    with open(Path('logger.json')) as f:
            logger_abi = json.load(f)

    logger_contract = w3.eth.contract(
        address=contract_address,
        abi=logger_abi
            )
    return logger_contract

def handle_new_log(event):
    io = StringIO((Web3.toJSON(event)))
    return json.load(io)
    

def handle_new_trade(event):
    io = StringIO((Web3.toJSON(event)))
    return json.load(io)
    


In [17]:
# Instantiate deployer and trader dictionary
deployer, trader_dict = load_deployer_contract()

deployer.address

'0x4971CE19A9d9f57941C4C3A00ffE69d06746073b'

In [20]:

# Create a filter to find new log contracts
new_log_filter = deployer.events.logCreated.createFilter(fromBlock="latest")

# Listen
print("Starting while loop...")
while True:
    # 1) Listen for new traders creating trading logs
    new_log_event = handle_new_log(new_log_filter.get_new_entries())
    if new_log_event:
        print("--------------------------")
        print((new_log_event[0]['args']))
        print("--------------------------")
        
        # add the information to a Grader object stored at that trader's ID variable
        new_log = new_log_event[0]['args']
        trader_dict[new_log['TraderId']]=Grader(new_log['TraderId'], new_log['newLogAddress'])
        trader_dict[new_log['TraderId']].load_contract()
            
    
    # loop through each trader and look for new trades at their log's contract address
    for key in trader_dict.keys():
        new_trade_event = handle_new_trade(trader_dict[key].new_trade_filter.get_new_entries())
        new_close_trade_event = handle_new_trade(trader_dict[key].close_trade_filter.get_new_entries())
        if new_trade_event:
            print("--------------------------")
            print((new_trade_event))
            print("--------------------------")
        if new_close_trade_event:
            new_close_trade_event[0]["TraderId"] = key 
            print("--------------------------")
            print((new_close_trade_event))
            print("--------------------------")  
                
    time.sleep(3)

Starting while loop...
--------------------------
[{'args': {'TraderId': 15, 'inputTraderAddress': '0x902A693035257b8f1e999E4a2414c450F5cB9E03', 'tradeNum': 1, 'inputOpen': True, 'inputSymbol': 'AAPL', 'inputSize': '500', 'inputEntryPriceEntryTime': '176 8/1/2022', 'optionsData': 'none'}, 'event': 'newTrade', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': '0x23823384836f39f9bedee8513228895f17965eade527b7075a4f10306930a8f5', 'address': '0xb306c57C3286d49cA4eBf0b90d75b6F4E13E5b6e', 'blockHash': '0x223cb24835ad977df5ec137dfb7edecc5826274beed0e7bd1ab71e2a0d2c036d', 'blockNumber': 109}]
--------------------------
--------------------------
[{'args': {'tradeNum': 1, 'exitPriceExitTime': '178, 8/1/2022'}, 'event': 'newCloseTrade', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': '0x1f4f2b68171acba53ab4b1acbd071d5d0d66a453d574d475af0abb9140a6aca2', 'address': '0xb306c57C3286d49cA4eBf0b90d75b6F4E13E5b6e', 'blockHash': '0x33a8d34efacd696278ea3769e6816dcb029ecfd5c5403995c3ff52c7

KeyboardInterrupt: 

In [11]:
trader_dict[13].close_trade_filter.__dict__

{'log_entry_formatter': <function get_event_data at 0x000001E7A0C081F8>,
 'eth_module': <web3.eth.Eth at 0x1e7a3a04d88>,
 'filter_id': '0x1d5',
 'callbacks': [],
 'filter_params': {'topics': ('0xa633b3cf9cf604b062687d6d28d6de58166d133b1709e63b10f3f893986f6292',),
  'fromBlock': 'latest',
  'toBlock': 'latest',
  'address': '0xd032b99d5863e39F777Fc40ED0276f84888DCAbD'},
 'data_filter_set': (('uint256', None), ('uint256', None), ('string', None)),
 'data_filter_set_function': <function match_fn at 0x000001E7A0C0DCA8>,
 'builder': <web3._utils.events.EventFilterBuilder at 0x1e7a3ad9908>}

In [45]:
new_log_filter.__dict__

{'log_entry_formatter': <function get_event_data at 0x00000162584D51F8>,
 'eth_module': <web3.eth.Eth at 0x1625b2d5d48>,
 'filter_id': '0x1b3',
 'callbacks': [],
 'filter_params': {'topics': ('0x3ccb4987690c8e37ec3a000310e45fbcef3730cf6068d27d5c688f294138287b',),
  'fromBlock': 'latest',
  'toBlock': 'latest',
  'address': '0x4971CE19A9d9f57941C4C3A00ffE69d06746073b'},
 'data_filter_set': (('uint256', None), ('address', None)),
 'data_filter_set_function': <function match_fn at 0x00000162584DACA8>,
 'builder': <web3._utils.events.EventFilterBuilder at 0x1625b664a48>}

In [48]:
trader_dict[key].new_trade_filter.__dict__

{'log_entry_formatter': <function get_event_data at 0x00000162584D51F8>,
 'eth_module': <web3.eth.Eth at 0x1625b2d5d48>,
 'filter_id': '0x1b5',
 'callbacks': [],
 'filter_params': {'topics': ('0xd31221654ebdc5f164b905fcf4b03959974549d0c7d04cf98383f5775b64b907',),
  'fromBlock': 'latest',
  'toBlock': 'latest',
  'address': '0x86E6A9885fee00874AcCD6903eD307716c9181DD'},
 'data_filter_set': (('uint256', None),
  ('address', None),
  ('uint256', None),
  ('bool', None),
  ('string', None),
  ('string', None),
  ('string', None),
  ('string', None)),
 'data_filter_set_function': <function match_fn at 0x00000162584DACA8>,
 'builder': <web3._utils.events.EventFilterBuilder at 0x1625b6c5ac8>}

In [39]:
trader_dict[2].__dict__


{'trader_id': 2,
 'log_address': '0xEe1Dcb4Ed69Ff562d0A546aa66F8d900A7961BC2',
 'contract': <web3._utils.datatypes.Contract at 0x1625b5e2948>,
 'new_trade_filter': <web3._utils.filters.LogFilter at 0x1625b57b248>}

In [13]:
for key in trader_dict.keys():
    print(trader_dict[key].log_address)

In [95]:
log_contract = load_contract_new(trader_dict[10].log_address)
event_filter = log_contract.events.newTrade.createFilter(fromBlock="latest")
event = handle_new_trade(event_filter.get_all_entries())
event

[{'args': {'TraderId': 10,
   'inputTraderAddress': '0x902A693035257b8f1e999E4a2414c450F5cB9E03',
   'tradeNum': 0,
   'inputOpen': True,
   'inputSymbol': 'MSFT',
   'inputSize': 100,
   'inputFractional_shares': 0,
   'inputEntryPrice': 270,
   'inputExitPrice': 0,
   'inputExpirationTimeStamp': 0,
   'inputStrike': 0,
   'inputIsCall': True},
  'event': 'newTrade',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': '0x81c3a64878b1715ec220e47912b20a19bb9be569b1fd5e84afdd08c5d447abd3',
  'address': '0xa51183B1aeBf945b99B9A669fBbF0A0a617d6008',
  'blockHash': '0xf37c55755000731c3fedd4338bd5408660fb0299963d922fcf8028ba57e2eec4',
  'blockNumber': 49}]

In [ ]:

contract = load_contract() 

event_filter = contract.events.newTrade.createFilter(fromBlock="latest")

while True:
    event = handle_new_trade(event_filter.get_new_entries())

    print((event))
    print("--------------------------")
    print(type(event))
    time.sleep(10)

[]
--------------------------
<class 'list'>


In [32]:
event

{'args': {'inputTraderAddress': '0x902A693035257b8f1e999E4a2414c450F5cB9E03',
  'TradeId': 6,
  'inputOpen': True,
  'inputSymbol': 'MSFT',
  'inputSize': 100,
  'inputFractional_shares': 0,
  'inputEntryPrice': 260,
  'inputExitPrice': 0,
  'inputExpirationTimeStamp': 0,
  'inputStrike': 0,
  'inputIsCall': True},
 'event': 'newTrade',
 'logIndex': 0,
 'transactionIndex': 0,
 'transactionHash': '0x4495bd80668a1894602a25eb47f6552d466de87881e8cbf1524ed930bd307287',
 'address': '0xD6e7211B6199C64C7fd3578761dCC4b14B2Ec12d',
 'blockHash': '0xb21554d0825169cc5cb963ef3bc709313f80a55e89ca7db53383618a51a39e67',
 'blockNumber': 35}

In [53]:
newLog = [{'args': {'TraderId': 5, 'newLogAddress': '0x7310FEB5415b50505C49B6c882BdaBc1782E9d60'}, 'event': 'logCreated', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': '0xe79708588ce0cd9efea795fc942a5df83b00707fd8281b1b8d2f2cb22b8a5a32', 'address': '0x65570CFF945218356882cE36F3bEE72aca300206', 'blockHash': '0xf30c42450e1614851d85d99226ebdd4c91f565409ec0e78485f7f894a835b32e', 'blockNumber': 42}]

In [60]:
new_log  = newLog[0]['args']
new_log

{'TraderId': 5, 'newLogAddress': '0x7310FEB5415b50505C49B6c882BdaBc1782E9d60'}

In [63]:
trader_dict = {}
trader_dict[new_log['TraderId']]=Grader(new_log['TraderId'],new_log['newLogAddress'])

In [76]:
trader_dict[8].log_address

'0x809Fb2c2E74f238dA88b58aBC598e8872195Ca58'

In [77]:
def load_contract_new(log_addr):
    contract_address = log_addr
    with open(Path('logger.json')) as f:
            logger_abi = json.load(f)

    logger_contract = w3.eth.contract(
    address=contract_address,
    abi=logger_abi
        )
    return logger_contract

In [78]:
ex_contract = load_contract_new(trader_dict[8].log_address)

In [79]:
ex_contract